taken from https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py

In [0]:
import keras
from keras.applications import VGG16
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential, Model
from sklearn.mixture import GaussianMixture

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(56, 56, 3))
conv_out = vgg_conv.get_layer('block2_conv2').output
avgpool = GlobalAveragePooling2D()(conv_out)
model = Model(inputs=vgg_conv.input, outputs=avgpool)

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os

# image folder
# path to model
# model_path = '/path/to/saved/model.h5'
# dimensions of images
img_width, img_height = 56, 56

# load the trained model
# model = load_model(model_path)
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# load all images into a list
def get_x_test(folder_path):
  images = []
  for img in os.listdir(folder_path):
      img = os.path.join(folder_path, img)
      img = image.load_img(img, target_size=(img_width, img_height))
      img = image.img_to_array(img)
      img = np.expand_dims(img, axis=0)
      images.append(img)

  # stack up images list to pass for prediction
  x_test = np.vstack(images)
  return x_test
# classes = model.predict(images) # model
# print(classes)


In [0]:
gmm = GaussianMixture(n_components=1)
# x_ok

In [0]:
from keras.datasets import mnist
import cv2
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_ok = x_train[y_train <= 2] # 6742 筆
# x_test = x_test[(y_test == 7) | (y_test == 1)] # 1135 筆 "1", 1028 筆 "7"
# y_test = y_test[(y_test == 7) | (y_test == 1)]
x_test = get_x_test(test_path)

def reshape_x(x):
    new_x = np.empty((len(x), 56, 56))
    for i, e in enumerate(x):
        new_x[i] = cv2.resize(e, (56, 56))

    new_x = np.expand_dims(new_x, axis=-1)
    new_x = np.repeat(new_x, 3, axis=-1)
    return new_x
  
x_ok = reshape_x(x_ok)
# x_test = reshape_x(x_test)

In [0]:
from keras.datasets import mnist
import cv2
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_ok = x_train[y_train <= 2] # 6742 筆
# x_test = x_test[(y_test == 7) | (y_test == 1)] # 1135 筆 "1", 1028 筆 "7"
# y_test = y_test[(y_test == 7) | (y_test == 1)]
x_test = get_x_test(test_path)

def reshape_x(x):
    new_x = np.empty((len(x), 56, 56, 3))
    for i, e in enumerate(x):
        if len(e.shape) <= 2:
          new_x[i] = np.repeat(np.expand_dims(cv2.resize(e, (56, 56)), axis=-1),3,axis=-1)
        else:
          new_x[i] = e
#     new_x = np.expand_dims(new_x, axis=-1)
#     new_x = np.repeat(new_x, 3, axis=-1)
    return new_x
  
x_ok = reshape_x(x_ok)
x_test = reshape_x(x_test)

In [52]:
features = model.predict(x_ok)
gmm.fit(features)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=1, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [54]:
OKscore = gmm.score_samples(features)
thred = OKscore.mean() - 3 * OKscore.std()

test_features = model.predict(x_test)
score = gmm.score_samples(test_features)

# print('normal accuracy: %.2f' % (len(score[(y_test <= 2) & (score > thred)]) / 1135))
# print('abnormal accuracy: %.2f' % (len(score[(y_test == 7) & (score < thred)]) / 1028))
# view raw

ValueError: ignored

In [0]:
plt.scatter(range(len(x_test)), score, c=['skyblue' if x <= 2 else 'pink' for x in y_test])
# plt.plotrange(len(x_test), [thred]*len(x_test), c='black')